In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
import ipywidgets as widgets
from IPython.display import display, clear_output
import random
from typing import List, Dict, Tuple
import difflib
import warnings
import json
import time
from matplotlib.patches import Polygon
warnings.filterwarnings('ignore')

class AdvancedPokemonTeamBuilder:
    def __init__(self, csv_path):
        """Initialize the Advanced Pokemon Team Builder with data from CSV."""
        self.df = self._load_data(csv_path)
        self._prepare_data()
        self._setup_competitive_tiers()
        self._setup_move_data()
        self._setup_evolution_data()
        print("Advanced Pokémon Team Builder initialized successfully!")

    def _load_data(self, csv_path):
        """Load the Pokemon dataset."""
        print(f"Loading data from {csv_path}...")
        df = pd.read_csv(csv_path)
        print(f"Loaded {len(df)} Pokémon successfully!")
        return df

    def _prepare_data(self):
        """Prepare the data for analysis."""
        # Clean the data
        self.df = self.df.fillna({
            'type2': 'None',
            'percentage_male': 50.0,
            'capture_rate': '45'
        })

        # Convert capture_rate to numeric
        self.df['capture_rate'] = self.df['capture_rate'].apply(
            lambda x: float(x.split()[0]) if isinstance(x, str) and ' ' in x else float(x)
        )

        # Create a lookup dictionary of Pokemon names to their data
        self.pokemon_lookup = {name.lower(): data for name, data in zip(self.df['name'], self.df.to_dict('records'))}

        # Create a list of all Pokemon names for reference
        self.all_pokemon_names = sorted(self.df['name'].tolist())

        # Create type effectiveness lookup
        self._create_type_effectiveness_lookup()

    def _create_type_effectiveness_lookup(self):
        """Create a lookup table for type effectiveness."""
        self.type_effectiveness = {}
        all_types = ['bug', 'dark', 'dragon', 'electric', 'fairy', 'fight', 'fire',
                     'flying', 'ghost', 'grass', 'ground', 'ice', 'normal',
                     'poison', 'psychic', 'rock', 'steel', 'water']

        # For each attacking type
        for attack_type in all_types:
            self.type_effectiveness[attack_type] = {}

            # For each defending type
            for defend_type in all_types:
                col_name = f'against_{defend_type}'

                # Find a Pokemon of this type to get effectiveness
                type_sample = self.df[self.df['type1'] == defend_type]
                if not type_sample.empty and col_name in self.df.columns:
                    # Convert the "against" value to actual multiplier (2.0 - value)
                    # The CSV has values like 0.5 meaning "2x effective", so we convert it
                    effectiveness = 2.0 - type_sample.iloc[0][f'against_{attack_type}']
                    self.type_effectiveness[attack_type][defend_type] = effectiveness
                else:
                    # Default to neutral if missing
                    self.type_effectiveness[attack_type][defend_type] = 1.0

    def _setup_competitive_tiers(self):
        """Set up competitive tiers for Pokemon analysis."""
        # Define tiers based on base_total stat (simplified approach)
        # In a real implementation, this would be based on usage stats or actual competitive tiers
        self.tiers = {
            'Uber': 600,  # Pokemon with base_total ≥ 600
            'OU': 550,    # Pokemon with base_total ≥ 550
            'UU': 500,    # Pokemon with base_total ≥ 500
            'RU': 450,    # Pokemon with base_total ≥ 450
            'NU': 400,    # Pokemon with base_total ≥ 400
            'PU': 0       # All other Pokemon
        }

        # Assign tiers to each Pokemon
        self.df['tier'] = 'PU'
        for tier, min_stat in self.tiers.items():
            self.df.loc[self.df['base_total'] >= min_stat, 'tier'] = tier

        # Break if a Pokemon is already assigned to a higher tier
        for tier_order, tier in enumerate(['Uber', 'OU', 'UU', 'RU', 'NU', 'PU']):
            mask = self.df['tier'] == tier
            for higher_tier in ['Uber', 'OU', 'UU', 'RU', 'NU', 'PU'][:tier_order]:
                mask = mask & (self.df['tier'] != higher_tier)
            self.df.loc[mask, 'tier'] = tier

    def _setup_move_data(self):
        """Set up move data for recommendations."""
        # Simplified move data since we don't have an API
        # Maps Pokemon types to recommended move types for coverage
        self.move_recommendations = {
            'bug': ['bug', 'flying', 'poison'],
            'dark': ['dark', 'ghost', 'fighting'],
            'dragon': ['dragon', 'fire', 'flying'],
            'electric': ['electric', 'flying', 'steel'],
            'fairy': ['fairy', 'flying', 'psychic'],
            'fight': ['fighting', 'rock', 'dark'],
            'fire': ['fire', 'flying', 'ground'],
            'flying': ['flying', 'normal', 'steel'],
            'ghost': ['ghost', 'dark', 'psychic'],
            'grass': ['grass', 'poison', 'bug'],
            'ground': ['ground', 'rock', 'steel'],
            'ice': ['ice', 'water', 'fighting'],
            'normal': ['normal', 'fighting', 'ground'],
            'poison': ['poison', 'dark', 'ground'],
            'psychic': ['psychic', 'fairy', 'fighting'],
            'rock': ['rock', 'ground', 'fighting'],
            'steel': ['steel', 'rock', 'electric'],
            'water': ['water', 'ice', 'ground']
        }

        # Maps stat preferences to move categories
        self.stat_to_move_category = {
            'attack': 'Physical',
            'sp_attack': 'Special'
        }

    def _setup_evolution_data(self):
        """Set up evolution data for recommendations."""
        # This is a simplified implementation
        # In a real implementation, this would use an API or complete evolution chains dataset

        # Example evolution chains (a very small subset for demonstration)
        self.evolution_chains = {
            'bulbasaur': ['ivysaur', 'venusaur'],
            'ivysaur': ['venusaur'],
            'charmander': ['charmeleon', 'charizard'],
            'charmeleon': ['charizard'],
            'squirtle': ['wartortle', 'blastoise'],
            'wartortle': ['blastoise'],
            'caterpie': ['metapod', 'butterfree'],
            'metapod': ['butterfree'],
            'weedle': ['kakuna', 'beedrill'],
            'kakuna': ['beedrill'],
            'pidgey': ['pidgeotto', 'pidgeot'],
            'pidgeotto': ['pidgeot'],
            'rattata': ['raticate'],
            'spearow': ['fearow'],
            'ekans': ['arbok'],
            'pichu': ['pikachu', 'raichu'],
            'pikachu': ['raichu'],
            'sandshrew': ['sandslash'],
            'nidoran-f': ['nidorina', 'nidoqueen'],
            'nidorina': ['nidoqueen'],
            'nidoran-m': ['nidorino', 'nidoking'],
            'nidorino': ['nidoking'],
            'cleffa': ['clefairy', 'clefable'],
            'clefairy': ['clefable'],
            'vulpix': ['ninetales'],
            'igglybuff': ['jigglypuff', 'wigglytuff'],
            'jigglypuff': ['wigglytuff'],
            'zubat': ['golbat', 'crobat'],
            'golbat': ['crobat'],
            'oddish': ['gloom', 'vileplume', 'bellossom'],
            'gloom': ['vileplume', 'bellossom'],
            'paras': ['parasect'],
            'venonat': ['venomoth'],
            'diglett': ['dugtrio'],
            'meowth': ['persian'],
            'psyduck': ['golduck'],
            'mankey': ['primeape'],
            'growlithe': ['arcanine'],
            'poliwag': ['poliwhirl', 'poliwrath', 'politoed'],
            'poliwhirl': ['poliwrath', 'politoed'],
            'abra': ['kadabra', 'alakazam'],
            'kadabra': ['alakazam'],
            'machop': ['machoke', 'machamp'],
            'machoke': ['machamp'],
            'bellsprout': ['weepinbell', 'victreebel'],
            'weepinbell': ['victreebel'],
            'tentacool': ['tentacruel'],
            'geodude': ['graveler', 'golem'],
            'graveler': ['golem'],
            'ponyta': ['rapidash'],
            'slowpoke': ['slowbro', 'slowking'],
            'magnemite': ['magneton', 'magnezone'],
            'magneton': ['magnezone'],
            'doduo': ['dodrio'],
            'seel': ['dewgong'],
            'grimer': ['muk'],
            'shellder': ['cloyster'],
            'gastly': ['haunter', 'gengar'],
            'haunter': ['gengar'],
            'drowzee': ['hypno'],
            'krabby': ['kingler'],
            'voltorb': ['electrode'],
            'exeggcute': ['exeggutor'],
            'cubone': ['marowak'],
            'tyrogue': ['hitmonlee', 'hitmonchan', 'hitmontop'],
            'lickitung': ['lickilicky'],
            'koffing': ['weezing'],
            'rhyhorn': ['rhydon', 'rhyperior'],
            'rhydon': ['rhyperior'],
            'chansey': ['blissey'],
            'horsea': ['seadra', 'kingdra'],
            'seadra': ['kingdra'],
            'goldeen': ['seaking'],
            'staryu': ['starmie'],
            'scyther': ['scizor'],
            'smoochum': ['jynx'],
            'elekid': ['electabuzz', 'electivire'],
            'electabuzz': ['electivire'],
            'magby': ['magmar', 'magmortar'],
            'magmar': ['magmortar'],
            'eevee': ['vaporeon', 'jolteon', 'flareon', 'espeon', 'umbreon', 'leafeon', 'glaceon', 'sylveon'],
            'omanyte': ['omastar'],
            'kabuto': ['kabutops'],
            'dratini': ['dragonair', 'dragonite'],
            'dragonair': ['dragonite'],
            'chikorita': ['bayleef', 'meganium'],
            'bayleef': ['meganium'],
            'cyndaquil': ['quilava', 'typhlosion'],
            'quilava': ['typhlosion'],
            'totodile': ['croconaw', 'feraligatr'],
            'croconaw': ['feraligatr'],
            'sentret': ['furret'],
            'hoothoot': ['noctowl'],
            'ledyba': ['ledian'],
            'spinarak': ['ariados'],
            'chinchou': ['lanturn'],
            'togepi': ['togetic', 'togekiss'],
            'togetic': ['togekiss'],
            'mareep': ['flaaffy', 'ampharos'],
            'flaaffy': ['ampharos'],
            'azurill': ['marill', 'azumarill'],
            'marill': ['azumarill'],
            'budew': ['roselia', 'roserade'],
            'roselia': ['roserade'],
            'hoppip': ['skiploom', 'jumpluff'],
            'skiploom': ['jumpluff'],
            'sunkern': ['sunflora'],
            'wooper': ['quagsire'],
            'wynaut': ['wobbuffet'],
            'pineco': ['forretress'],
            'snubbull': ['granbull']
        }

        # Create a reverse lookup for evolutions
        self.pre_evolutions = {}
        for base, evos in self.evolution_chains.items():
            for evo in evos:
                self.pre_evolutions[evo] = base

    def get_pokemon_by_name(self, name):
        """Get a Pokemon's data by name, with fuzzy matching."""
        if not name or name.strip() == "":
            return None

        name_lower = name.lower().strip()

        # Exact match
        if name_lower in self.pokemon_lookup:
            return self.pokemon_lookup[name_lower]

        # Fuzzy match using difflib
        matches = difflib.get_close_matches(name_lower, self.pokemon_lookup.keys(), n=1, cutoff=0.6)
        if matches:
            match = matches[0]
            print(f"Using '{match.title()}' instead of '{name}'")
            return self.pokemon_lookup[match]

        # Close match (fallback)
        for pokemon_name in self.pokemon_lookup.keys():
            if name_lower in pokemon_name or pokemon_name in name_lower:
                print(f"Using '{pokemon_name.title()}' instead of '{name}'")
                return self.pokemon_lookup[pokemon_name]

        # If no match found
        print(f"Warning: Pokémon '{name}' not found in database.")
        return None

    def pokemon_to_dict(self, pokemon):
        """Convert a Pokemon dataframe row to a dictionary format."""
        if pokemon is None:
            return None

        return {
            'name': pokemon['name'],
            'types': [pokemon['type1']] if pokemon['type2'] == 'None' else [pokemon['type1'], pokemon['type2']],
            'stats': {
                'hp': pokemon['hp'],
                'attack': pokemon['attack'],
                'defense': pokemon['defense'],
                'sp_attack': pokemon['sp_attack'],
                'sp_defense': pokemon['sp_defense'],
                'speed': pokemon['speed'],
                'total': pokemon['base_total']
            },
            'abilities': str(pokemon['abilities']).split(',') if isinstance(pokemon['abilities'], str) else [],
            'tier': pokemon['tier'] if 'tier' in pokemon else 'Unknown'
        }

    def generate_counter_team(self, opponent_team, size=6):
        """Generate a team to counter the specified opponent team."""
        if not opponent_team:
            print("No opponent team provided.")
            return []

        print(f"Generating counter team for {len(opponent_team)} Pokémon...")

        # Extract opponent types
        opponent_types = set()
        for pokemon in opponent_team:
            if pokemon is None:
                continue
            for type_name in pokemon['types']:
                opponent_types.add(type_name)

        print(f"Opponent types: {', '.join(opponent_types)}")

        # Calculate effectiveness scores against opponent types
        all_pokemon = []
        for _, pokemon in self.df.iterrows():
            # Calculate average effectiveness against opponent types
            effectiveness_score = 0
            for type_name in opponent_types:
                type_col = f"against_{type_name.lower()}"
                if type_col in self.df.columns:
                    effectiveness_score += 2.0 - pokemon[type_col]  # Invert so higher is better

            effectiveness_score /= len(opponent_types) if opponent_types else 1

            # Consider Pokemon's stats too
            stat_score = pokemon['base_total'] / 600.0  # Normalize

            # Combined score (higher is better)
            combined_score = (effectiveness_score * 0.7) + (stat_score * 0.3)
            all_pokemon.append((pokemon, combined_score))

        # Sort by score and get the top Pokemon
        all_pokemon.sort(key=lambda x: x[1], reverse=True)

        # Build a diverse counter team
        counter_team = []
        selected_types = set()

        for pokemon, score in all_pokemon:
            if len(counter_team) >= size:
                break

            # Skip if already in team
            if any(p['name'] == pokemon['name'] for p in counter_team if p is not None):
                continue

            # Add to counter team
            counter_pokemon = self.pokemon_to_dict(pokemon)
            if counter_pokemon:
                counter_team.append(counter_pokemon)

                # Update selected types
                for t in counter_pokemon['types']:
                    selected_types.add(t)

        return counter_team

    def display_team(self, team, title="Team Analysis"):
        """Display a team with Pokémon information."""
        if not team:
            print("No team to display.")
            return

        valid_team = [p for p in team if p is not None]
        if not valid_team:
            print("No valid Pokémon in team.")
            return

        print(f"\n{title}:")
        for i, pokemon in enumerate(valid_team, 1):
            print(f"{i}. {pokemon['name']} - Types: {', '.join(pokemon['types'])}")
            print(f"   Stats: HP {pokemon['stats']['hp']}, Atk {pokemon['stats']['attack']}, Def {pokemon['stats']['defense']}")
            print(f"          Sp.Atk {pokemon['stats']['sp_attack']}, Sp.Def {pokemon['stats']['sp_defense']}, Spd {pokemon['stats']['speed']}")
            print(f"   Abilities: {', '.join(pokemon['abilities'])}")
            if 'tier' in pokemon:
                print(f"   Competitive Tier: {pokemon['tier']}")
            print()

        # Create visualizations
        self._visualize_team_stats(valid_team, title)
        self._visualize_team_types(valid_team, title)

    def _visualize_team_stats(self, team, title):
        """Visualize team stats using a radar chart."""
        # Create figure with polar projection for radar chart
        fig = plt.figure(figsize=(10, 6))
        ax = fig.add_subplot(111, polar=True)

        # Define stat categories
        stats = ['HP', 'Attack', 'Defense', 'Sp. Atk', 'Sp. Def', 'Speed']

        # Calculate team averages
        team_avg = {
            'HP': sum(p['stats']['hp'] for p in team) / len(team),
            'Attack': sum(p['stats']['attack'] for p in team) / len(team),
            'Defense': sum(p['stats']['defense'] for p in team) / len(team),
            'Sp. Atk': sum(p['stats']['sp_attack'] for p in team) / len(team),
            'Sp. Def': sum(p['stats']['sp_defense'] for p in team) / len(team),
            'Speed': sum(p['stats']['speed'] for p in team) / len(team)
        }

        # Comparison to average Pokemon
        avg_stats = {
            'HP': self.df['hp'].mean(),
            'Attack': self.df['attack'].mean(),
            'Defense': self.df['defense'].mean(),
            'Sp. Atk': self.df['sp_attack'].mean(),
            'Sp. Def': self.df['sp_defense'].mean(),
            'Speed': self.df['speed'].mean()
        }

        # Number of variables
        N = len(stats)

        # Create angle for each stat
        angles = [n / float(N) * 2 * np.pi for n in range(N)]
        angles += angles[:1]  # Close the loop

        # Set radar chart parameters
        ax.set_theta_offset(np.pi / 2)
        ax.set_theta_direction(-1)
        ax.set_xticks(angles[:-1])
        ax.set_xticklabels(stats)

        # Plot data
        team_values = [team_avg[s] for s in stats]
        team_values += team_values[:1]  # Close the loop

        avg_values = [avg_stats[s] for s in stats]
        avg_values += avg_values[:1]  # Close the loop

        # Plot values
        ax.plot(angles, team_values, 'b-', linewidth=2, label=f'{title}')
        ax.fill(angles, team_values, 'b', alpha=0.1)

        ax.plot(angles, avg_values, 'r-', linewidth=2, label='Average Pokémon')
        ax.fill(angles, avg_values, 'r', alpha=0.1)

        # Add legend and title
        plt.legend(loc='upper right', bbox_to_anchor=(0.1, 0.1))
        plt.title(f"{title} - Stat Comparison", size=15, y=1.1)

        plt.tight_layout()
        plt.show()

    def _visualize_team_types(self, team, title):
        """Visualize team type distribution."""
        team_types = []
        for pokemon in team:
            team_types.extend(pokemon['types'])

        type_counts = {}
        for t in team_types:
            type_counts[t] = type_counts.get(t, 0) + 1

        plt.figure(figsize=(10, 6))
        colors = plt.cm.viridis(np.linspace(0.1, 0.9, len(type_counts)))
        plt.pie(type_counts.values(), labels=type_counts.keys(), autopct='%1.1f%%',
               startangle=140, colors=colors)
        plt.axis('equal')
        plt.title(f"{title} - Type Distribution")
        plt.tight_layout()
        plt.show()

    def analyze_team_weaknesses(self, team):
        """Analyze weaknesses of a team."""
        if not team:
            print("No team to analyze.")
            return

        valid_team = [p for p in team if p is not None]
        if not valid_team:
            print("No valid Pokémon in team.")
            return

        print("\n📊 Team Weakness Analysis:")

        # Extract team types
        team_types = []
        for pokemon in valid_team:
            team_types.extend(pokemon['types'])

        # Calculate team defense against each type
        type_weaknesses = {}
        attacking_types = ['bug', 'dark', 'dragon', 'electric', 'fairy', 'fight', 'fire',
                           'flying', 'ghost', 'grass', 'ground', 'ice', 'normal',
                           'poison', 'psychic', 'rock', 'steel', 'water']

        for attack_type in attacking_types:
            effectiveness = []
            for pokemon in valid_team:
                # Calculate effectiveness against this Pokemon
                pokemon_effectiveness = 1.0
                for defense_type in pokemon['types']:
                    defense_type_lower = defense_type.lower()
                    # Some types might be abbreviated in the data
                    if defense_type_lower == 'fighting':
                        defense_type_lower = 'fight'

                    # Get effectiveness multiplier
                    type_col = f"against_{attack_type}"
                    if defense_type_lower in self.df['type1'].values and type_col in self.df.columns:
                        # Find a sample Pokemon of this defensive type
                        type_sample = self.df[self.df['type1'] == defense_type_lower]
                        if not type_sample.empty:
                            # Get the effectiveness value and convert it
                            if attack_type in self.df.columns:
                                type_effect = 2.0 - type_sample.iloc[0][type_col]
                                pokemon_effectiveness *= type_effect

                effectiveness.append(pokemon_effectiveness)

            # Calculate average effectiveness against the team
            avg_effectiveness = sum(effectiveness) / len(effectiveness)
            type_weaknesses[attack_type] = avg_effectiveness

        # Sort weaknesses
        sorted_weaknesses = sorted(type_weaknesses.items(), key=lambda x: x[1], reverse=True)

        # Display major weaknesses
        major_weaknesses = [t for t, e in sorted_weaknesses if e > 1.2]
        if major_weaknesses:
            print("Major Team Weaknesses:")
            for weakness in major_weaknesses:
                print(f"- {weakness.capitalize()} (effectiveness: {type_weaknesses[weakness]:.2f}x)")

            # Recommend counters
            print("\nPokémon that could exploit these weaknesses:")
            potential_threats = []
            for _, pokemon in self.df.iterrows():
                # Check if Pokemon has one of the weakness types
                if pokemon['type1'] in major_weaknesses or (pokemon['type2'] != 'None' and pokemon['type2'] in major_weaknesses):
                    threat_score = pokemon['base_total'] / 600.0  # Normalize by max stat
                    potential_threats.append((pokemon['name'], threat_score))

            # Sort and display top threats
            potential_threats.sort(key=lambda x: x[1], reverse=True)
            for threat, _ in potential_threats[:5]:
                print(f"- {threat}")

            # Suggest changes
            print("\nSuggested Team Adjustments:")
            for weakness in major_weaknesses[:3]:  # Top 3 weaknesses
                # Find Pokemon that resist this type
                resistant_types = []
                for t, effectiveness in self.type_effectiveness[weakness].items():
                    if effectiveness < 1.0:
                        resistant_types.append(t)

                if resistant_types:
                    print(f"- Add a {'/'.join(resistant_types[:2])} type Pokémon to counter {weakness.capitalize()} weakness")
        else:
            print("No major team weaknesses found. Your team has good defensive coverage!")

        # Calculate team coverage - what types the team can hit effectively
        print("\nOffensive Coverage Analysis:")
        coverage = set()
        for pokemon in valid_team:
            for p_type in pokemon['types']:
                for defending_type, effectiveness in self.type_effectiveness.get(p_type.lower(), {}).items():
                    if effectiveness > 1.0:
                        coverage.add(defending_type)

        missing_coverage = set(attacking_types) - coverage
        if missing_coverage:
            print(f"Types your team may struggle to hit effectively: {', '.join(missing_coverage)}")
        else:
            print("Excellent offensive coverage! Your team can hit all types effectively.")

    def rate_team(self, team):
        """Rate a team on various metrics."""
        if not team:
            print("No team to rate.")
            return

        valid_team = [p for p in team if p is not None]
        if not valid_team:
            print("No valid Pokémon in team.")
            return

        print("\n🏆 Team Rating System:")

        # Calculate various metrics
        avg_stats = {
            'hp': sum(p['stats']['hp'] for p in valid_team) / len(valid_team),
            'attack': sum(p['stats']['attack'] for p in valid_team) / len(valid_team),
            'defense': sum(p['stats']['defense'] for p in valid_team) / len(valid_team),
            'sp_attack': sum(p['stats']['sp_attack'] for p in valid_team) / len(valid_team),
            'sp_defense': sum(p['stats']['sp_defense'] for p in valid_team) / len(valid_team),
            'speed': sum(p['stats']['speed'] for p in valid_team) / len(valid_team),
            'total': sum(p['stats']['total'] for p in valid_team) / len(valid_team)
        }

        # Calculate ratings (0-100 scale)
        ratings = {
            'Offensive Power': min(100, (avg_stats['attack'] + avg_stats['sp_attack']) / 2 / 1.7),
            'Defensive Capability': min(100, (avg_stats['defense'] + avg_stats['sp_defense'] + avg_stats['hp']/2) / 3 / 1.5),
            'Speed': min(100, avg_stats['speed'] / 1.3),
            'Type Diversity': min(100, self._calculate_type_diversity(valid_team) * 100),
            'Stat Balance': min(100, self._calculate_stat_balance(valid_team) * 100),
            'Tier Rating': min(100, self._calculate_tier_rating(valid_team) * 100)
        }

        # Calculate overall score
        overall_score = sum(ratings.values()) / len(ratings)

        # Display ratings
        print(f"Team Rating (0-100 scale):")
        for category, rating in ratings.items():
            stars = self._rating_to_stars(rating)
            print(f"- {category}: {rating:.1f}/100 {stars}")

        # Display overall score
        stars = self._rating_to_stars(overall_score)
        print(f"\nOverall Team Score: {overall_score:.1f}/100 {stars}")

        # Visualize ratings
        self._visualize_team_ratings(ratings)

        # Give qualitative assessment
        self._provide_rating_insights(ratings, valid_team)

    def _rating_to_stars(self, rating):
        """Convert a rating to a star representation."""
        full_stars = int(rating / 20)
        return "★" * full_stars + "☆" * (5 - full_stars)

    def _calculate_type_diversity(self, team):
        """Calculate type diversity score."""
        team_types = set()
        for pokemon in team:
            team_types.update(pokemon['types'])

        # Score based on number of unique types (max 18 types)
        return min(1.0, len(team_types) / 12.0)  # 12 is a good target for 6 Pokémon

    def _calculate_stat_balance(self, team):
        """Calculate stat balance score."""
        stats = ['hp', 'attack', 'defense', 'sp_attack', 'sp_defense', 'speed']
        stat_values = {stat: [] for stat in stats}

        for pokemon in team:
            stat_values['hp'].append(pokemon['stats']['hp'])
            stat_values['attack'].append(pokemon['stats']['attack'])
            stat_values['defense'].append(pokemon['stats']['defense'])
            stat_values['sp_attack'].append(pokemon['stats']['sp_attack'])
            stat_values['sp_defense'].append(pokemon['stats']['sp_defense'])
            stat_values['speed'].append(pokemon['stats']['speed'])

        # Calculate coefficient of variation for each stat (lower is better)
        stat_balance = 0
        for stat in stats:
            mean = np.mean(stat_values[stat])
            std = np.std(stat_values[stat])
            if mean > 0:
                cv = std / mean  # Coefficient of variation
                stat_balance += (1 - min(cv, 1))  # Lower CV means better balance

        return stat_balance / len(stats)  # Normalize

    def _calculate_tier_rating(self, team):
        """Calculate competitive tier rating."""
        tier_values = {
            'Uber': 1.0,
            'OU': 0.9,
            'UU': 0.75,
            'RU': 0.6,
            'NU': 0.4,
            'PU': 0.2,
            'Unknown': 0.1
        }

        # Calculate average tier value
        avg_tier = sum(tier_values.get(p.get('tier', 'Unknown'), 0.1) for p in team) / len(team)

        return avg_tier

    def _visualize_team_ratings(self, ratings):
        """Visualize team ratings as a radar chart."""
        categories = list(ratings.keys())
        values = [ratings[cat] for cat in categories]

        # Normalize values
        values = [v / 100.0 for v in values]

        # Create radar chart
        fig = plt.figure(figsize=(10, 8))
        ax = fig.add_subplot(111, polar=True)

        # Set number of sides of the polygon
        N = len(categories)

        # Create angle for each category
        angles = [n / float(N) * 2 * np.pi for n in range(N)]
        angles += angles[:1]  # Close the loop

        # Add values for each category, closing the loop
        values += values[:1]

        # Set radar chart parameters
        ax.set_theta_offset(np.pi / 2)
        ax.set_theta_direction(-1)
        ax.set_xticks(angles[:-1])
        ax.set_xticklabels(categories)

        # Draw polygon and fill
        ax.plot(angles, values, 'b-', linewidth=2)
        ax.fill(angles, values, 'b', alpha=0.2)

        # Set y-axis limits
        ax.set_ylim(0, 1)

        # Add grid lines and labels
        for i in [0.2, 0.4, 0.6, 0.8, 1.0]:
            ax.text(np.pi/2, i+0.01, f"{int(i*100)}", ha='center')

        plt.title("Team Rating Radar Chart", size=15)
        plt.tight_layout()
        plt.show()

    def _provide_rating_insights(self, ratings, team):
        """Provide insights based on team ratings."""
        print("\nTeam Insights:")

        # Check for low ratings
        low_ratings = [cat for cat, rating in ratings.items() if rating < 60]
        if low_ratings:
            print(f"Areas needing improvement: {', '.join(low_ratings)}")

            # Provide specific advice for each low area
            for area in low_ratings:
                if area == 'Offensive Power':
                    print("- To improve Offensive Power: Add Pokémon with higher Attack or Special Attack stats")
                elif area == 'Defensive Capability':
                    print("- To improve Defensive Capability: Add Pokémon with higher HP, Defense, or Special Defense")
                elif area == 'Speed':
                    print("- To improve Speed: Add faster Pokémon or those with priority moves")
                elif area == 'Type Diversity':
                    print("- To improve Type Diversity: Add Pokémon with different types than your current team")
                elif area == 'Stat Balance':
                    print("- To improve Stat Balance: Add Pokémon that complement your team's weakest stats")
                elif area == 'Tier Rating':
                    print("- To improve Tier Rating: Add Pokémon from higher competitive tiers")
        else:
            print("No major weaknesses in your team ratings. Well done!")

        # Identify team style
        offensive = ratings['Offensive Power'] > 70
        defensive = ratings['Defensive Capability'] > 70
        fast = ratings['Speed'] > 70

        if offensive and fast and not defensive:
            print("\nTeam Style: Hyper Offense - Your team excels at hitting hard and fast")
        elif offensive and defensive and not fast:
            print("\nTeam Style: Bulky Offense - Your team combines offensive power with defensive staying power")
        elif defensive and fast and not offensive:
            print("\nTeam Style: Fast Stall - Your team uses speed and defense to wear down opponents")
        elif defensive and not offensive and not fast:
            print("\nTeam Style: Wall - Your team excels at taking hits and outlasting opponents")
        elif offensive and not defensive and not fast:
            print("\nTeam Style: Glass Cannon - Your team hits hard but might struggle to take hits")
        elif fast and not offensive and not defensive:
            print("\nTeam Style: Speedy Support - Your team is fast but might lack definitive offensive or defensive presence")
        else:
            print("\nTeam Style: Balanced - Your team has good overall balance")

    def recommend_moves(self, team):
        """Recommend moves for each Pokemon in the team."""
        if not team:
            print("No team to analyze.")
            return

        valid_team = [p for p in team if p is not None]
        if not valid_team:
            print("No valid Pokémon in team.")
            return

        print("\n🎯 Move Recommendations:")

        # Extract team types to avoid redundant coverage
        team_offensive_types = set()
        for pokemon in valid_team:
            team_offensive_types.update(pokemon['types'])

        for i, pokemon in enumerate(valid_team, 1):
            print(f"{i}. {pokemon['name']} ({'/'.join(pokemon['types'])}):")

            # Identify offensive stat preference
            physical = pokemon['stats']['attack'] > pokemon['stats']['sp_attack']
            category = "Physical" if physical else "Special"

            # Recommend STAB moves
            print(f"   STAB Moves ({category}):")
            for type_name in pokemon['types']:
                print(f"   - A {type_name}-type {category.lower()} move")

            # Recommend coverage moves
            coverage_recommendations = []

            # Get recommended coverage types from our simplified data
            for type_name in pokemon['types']:
                type_lower = type_name.lower()
                # Some types might need normalization
                if type_lower == 'fighting':
                    type_lower = 'fight'

                if type_lower in self.move_recommendations:
                    for cover_type in self.move_recommendations[type_lower]:
                        # Skip if already has STAB
                        if cover_type.capitalize() in pokemon['types']:
                            continue
                        # Skip if another team member has STAB of this type
                        if cover_type.capitalize() in team_offensive_types and cover_type.capitalize() not in pokemon['types']:
                            continue

                        coverage_recommendations.append(cover_type)

            # Get unique coverage recommendations
            coverage_recommendations = list(set(coverage_recommendations))

            if coverage_recommendations:
                print(f"   Coverage Moves:")
                for j, cover_type in enumerate(coverage_recommendations[:2], 1):  # Recommend top 2
                    print(f"   - A {cover_type}-type {category.lower()} move")

            # Recommend utility move
            print(f"   Utility Move Options:")

            # Suggest utility based on stats and types
            if pokemon['stats']['speed'] > 90:
                print(f"   - A status move (e.g., Thunder Wave, Will-O-Wisp)")
            elif pokemon['stats']['defense'] > 90 or pokemon['stats']['sp_defense'] > 90:
                print(f"   - A recovery move (e.g., Recover, Synthesis)")
            else:
                print(f"   - A setup move (e.g., Swords Dance, Nasty Plot)")

            print()

    def analyze_competitive_tiers(self, team):
        """Analyze the competitive viability of a team."""
        if not team:
            print("No team to analyze.")
            return

        valid_team = [p for p in team if p is not None]
        if not valid_team:
            print("No valid Pokémon in team.")
            return

        print("\n🏅 Competitive Tier Analysis:")

        # Count Pokemon in each tier
        tier_counts = {}
        for pokemon in valid_team:
            tier = pokemon.get('tier', 'Unknown')
            tier_counts[tier] = tier_counts.get(tier, 0) + 1

        # Display tier distribution
        print("Team Tier Distribution:")
        for tier in ['Uber', 'OU', 'UU', 'RU', 'NU', 'PU', 'Unknown']:
            if tier in tier_counts:
                print(f"- {tier}: {tier_counts[tier]} Pokémon")

        # Calculate team viability score
        tier_values = {
            'Uber': 100,
            'OU': 90,
            'UU': 75,
            'RU': 60,
            'NU': 40,
            'PU': 20,
            'Unknown': 10
        }

        viability_score = sum(tier_values.get(p.get('tier', 'Unknown'), 10) for p in valid_team) / len(valid_team)

        # Display viability assessment
        print(f"\nTeam Competitive Viability Score: {viability_score:.1f}/100")

        if viability_score >= 90:
            print("Excellent competitive viability! Your team is suited for high-level play.")
        elif viability_score >= 75:
            print("Good competitive viability. Your team should perform well in most competitive environments.")
        elif viability_score >= 60:
            print("Moderate competitive viability. Your team may struggle against top-tier teams.")
        else:
            print("Limited competitive viability. Consider replacing some lower-tier Pokémon for competitive play.")

        # Identify replacements for lower tier Pokemon
        if viability_score < 75:
            print("\nSuggested Replacements:")

            lower_tier_pokemon = [(i, p) for i, p in enumerate(valid_team)
                                 if p.get('tier', 'Unknown') not in ['Uber', 'OU', 'UU']]

            for i, pokemon in lower_tier_pokemon:
                # Find similar but higher tier Pokemon
                similar_pokemon = []

                for _, candidate in self.df.iterrows():
                    # Skip if not a higher tier
                    if candidate['tier'] not in ['Uber', 'OU', 'UU']:
                        continue

                    # Check if similar type
                    type_match = (candidate['type1'] in pokemon['types']) or \
                                (candidate['type2'] != 'None' and candidate['type2'] in pokemon['types'])

                    if type_match:
                        similar_pokemon.append(candidate)

                # Sort by base_total and get top 2
                similar_pokemon.sort(key=lambda x: x['base_total'], reverse=True)
                top_similar = similar_pokemon[:2]

                if top_similar:
                    print(f"Instead of {pokemon['name']} ({'/'.join(pokemon['types'])}), consider:")
                    for j, replacement in enumerate(top_similar, 1):
                        replacement_types = [replacement['type1']]
                        if replacement['type2'] != 'None':
                            replacement_types.append(replacement['type2'])
                        print(f"  {j}. {replacement['name']} ({'/'.join(replacement_types)}) - Tier: {replacement['tier']}")

        # Visualize tier distribution
        self._visualize_tier_distribution(tier_counts)

    def _visualize_tier_distribution(self, tier_counts):
        """Visualize tier distribution."""
        tiers = ['Uber', 'OU', 'UU', 'RU', 'NU', 'PU', 'Unknown']
        counts = [tier_counts.get(tier, 0) for tier in tiers]

        # Create bar chart
        plt.figure(figsize=(10, 6))
        colors = plt.cm.viridis(np.linspace(0.8, 0.2, len(tiers)))
        plt.bar(tiers, counts, color=colors)
        plt.title('Team Competitive Tier Distribution')
        plt.xlabel('Competitive Tier')
        plt.ylabel('Number of Pokémon')
        plt.ylim(0, max(counts) + 1)

        # Add value labels
        for i, count in enumerate(counts):
            if count > 0:
                plt.text(i, count + 0.1, str(count), ha='center')

        plt.tight_layout()
        plt.show()

    def suggest_evolutions(self, team):
        """Suggest evolution paths for the team."""
        if not team:
            print("No team to analyze.")
            return

        valid_team = [p for p in team if p is not None]
        if not valid_team:
            print("No valid Pokémon in team.")
            return

        print("\n🌱 Team Evolution Planner:")

        evolvable_pokemon = []
        for i, pokemon in enumerate(valid_team):
            name_lower = pokemon['name'].lower()

            # Check if Pokemon can evolve
            if name_lower in self.evolution_chains:
                evolvable_pokemon.append((i, pokemon, self.evolution_chains[name_lower]))

        if not evolvable_pokemon:
            print("None of your team members can evolve further.")
            return

        print(f"Found {len(evolvable_pokemon)} team members that can evolve:")

        for i, pokemon, evolutions in evolvable_pokemon:
            print(f"\n{i+1}. {pokemon['name']} can evolve into:")

            for j, evolution in enumerate(evolutions, 1):
                # Get evolution data
                evolution_data = self.get_pokemon_by_name(evolution)
                if evolution_data:
                    evolution_dict = self.pokemon_to_dict(evolution_data)

                    # Calculate stat improvements
                    stat_improvements = {
                        'hp': evolution_dict['stats']['hp'] - pokemon['stats']['hp'],
                        'attack': evolution_dict['stats']['attack'] - pokemon['stats']['attack'],
                        'defense': evolution_dict['stats']['defense'] - pokemon['stats']['defense'],
                        'sp_attack': evolution_dict['stats']['sp_attack'] - pokemon['stats']['sp_attack'],
                        'sp_defense': evolution_dict['stats']['sp_defense'] - pokemon['stats']['sp_defense'],
                        'speed': evolution_dict['stats']['speed'] - pokemon['stats']['speed'],
                        'total': evolution_dict['stats']['total'] - pokemon['stats']['total']
                    }

                    # Format types
                    pokemon_types = '/'.join(pokemon['types'])
                    evolution_types = '/'.join(evolution_dict['types'])

                    print(f"   {j}. {evolution.title()} ({evolution_types})")
                    print(f"      Stat Changes: HP +{stat_improvements['hp']}, "
                          f"Atk +{stat_improvements['attack']}, "
                          f"Def +{stat_improvements['defense']}, "
                          f"SpA +{stat_improvements['sp_attack']}, "
                          f"SpD +{stat_improvements['sp_defense']}, "
                          f"Spe +{stat_improvements['speed']}")
                    print(f"      Total Stat Increase: +{stat_improvements['total']}")

                    # Show type changes
                    if pokemon_types != evolution_types:
                        print(f"      Type Change: {pokemon_types} → {evolution_types}")

                    # Show competitive tier change
                    if 'tier' in pokemon and 'tier' in evolution_dict and pokemon['tier'] != evolution_dict['tier']:
                        print(f"      Tier Change: {pokemon['tier']} → {evolution_dict['tier']}")

        # Suggest team after all evolutions
        evolved_team = valid_team.copy()
        for i, pokemon, evolutions in evolvable_pokemon:
            if evolutions:  # Get the final evolution
                final_evolution = evolutions[-1]
                evolution_data = self.get_pokemon_by_name(final_evolution)
                if evolution_data:
                    evolved_team[i] = self.pokemon_to_dict(evolution_data)

        print("\nTeam Stats After All Evolutions:")
        before_total = sum(p['stats']['total'] for p in valid_team)
        after_total = sum(p['stats']['total'] for p in evolved_team)

        print(f"Current Team Total Stats: {before_total}")
        print(f"Evolved Team Total Stats: {after_total}")
        print(f"Stat Improvement: +{after_total - before_total} ({((after_total/before_total)-1)*100:.1f}%)")

        # Analyze team balance before and after evolution
        before_balance = self._calculate_stat_balance(valid_team)
        after_balance = self._calculate_stat_balance(evolved_team)

        print(f"Team Balance Score: {before_balance*100:.1f}% → {after_balance*100:.1f}%")

        if after_balance > before_balance:
            print("Evolutions will improve your team's stat balance!")
        elif after_balance < before_balance:
            print("Note: Evolutions might slightly reduce your team's stat balance, but the overall power increase is worth it.")
        else:
            print("Evolutions will maintain your team's current stat balance.")

    def simulate_battle(self, team1, team2):
        """Simulate a simplified battle between two teams."""
        if not team1 or not team2:
            print("Need two valid teams to simulate a battle.")
            return

        valid_team1 = [p for p in team1 if p is not None]
        valid_team2 = [p for p in team2 if p is not None]

        if not valid_team1 or not valid_team2:
            print("Both teams must have at least one valid Pokémon.")
            return

        print("\n⚔️ Battle Simulation:")
        print(f"Team 1 ({len(valid_team1)} Pokémon) vs Team 2 ({len(valid_team2)} Pokémon)")

        # Simplified battle simulation - each Pokemon battles once
        # This is a very simplified model and doesn't account for actual moves,
        # abilities, items, switching, etc.

        team1_remaining = len(valid_team1)
        team2_remaining = len(valid_team2)

        # Sort both teams by speed for battle order
        valid_team1.sort(key=lambda p: p['stats']['speed'], reverse=True)
        valid_team2.sort(key=lambda p: p['stats']['speed'], reverse=True)

        # Print team lineups
        print("\nTeam 1 Lineup:")
        for i, pokemon in enumerate(valid_team1, 1):
            print(f"{i}. {pokemon['name']} (HP: {pokemon['stats']['hp']}, Speed: {pokemon['stats']['speed']})")

        print("\nTeam 2 Lineup:")
        for i, pokemon in enumerate(valid_team2, 1):
            print(f"{i}. {pokemon['name']} (HP: {pokemon['stats']['hp']}, Speed: {pokemon['stats']['speed']})")

        print("\nBattle Start!\n")

        # Simulate the battle
        turn = 1
        team1_index = 0
        team2_index = 0

        # Battle log
        battle_log = []

        # Team scores
        team1_score = 0
        team2_score = 0

        while team1_remaining > 0 and team2_remaining > 0 and turn <= 100:  # Limit to 100 turns
            print(f"Turn {turn}:")

            # Current active Pokemon
            current_team1 = valid_team1[team1_index]
            current_team2 = valid_team2[team2_index]

            print(f"Team 1's {current_team1['name']} vs Team 2's {current_team2['name']}")

            # Calculate current HP (simplified)
            current_team1_hp = current_team1['stats']['hp']
            current_team2_hp = current_team2['stats']['hp']

            # Determine who goes first based on Speed
            if current_team1['stats']['speed'] >= current_team2['stats']['speed']:
                first = current_team1
                second = current_team2
                first_team = 1
                second_team = 2
            else:
                first = current_team2
                second = current_team1
                first_team = 2
                second_team = 1

            # First Pokemon attacks
            damage = self._calculate_damage(first, second)

            if first_team == 1:
                current_team2_hp -= damage
                log_entry = f"Team 1's {first['name']} attacks for {damage} damage!"
            else:
                current_team1_hp -= damage
                log_entry = f"Team 2's {first['name']} attacks for {damage} damage!"

            battle_log.append(log_entry)
            print(log_entry)

            # Check if the second Pokemon fainted
            if (first_team == 1 and current_team2_hp <= 0) or (first_team == 2 and current_team1_hp <= 0):
                if first_team == 1:
                    log_entry = f"Team 2's {second['name']} fainted!"
                    team2_remaining -= 1
                    team2_index = (team2_index + 1) % len(valid_team2)
                    team1_score += 1
                else:
                    log_entry = f"Team 1's {second['name']} fainted!"
                    team1_remaining -= 1
                    team1_index = (team1_index + 1) % len(valid_team1)
                    team2_score += 1

                battle_log.append(log_entry)
                print(log_entry)
            else:
                # Second Pokemon attacks if it didn't faint
                damage = self._calculate_damage(second, first)

                if second_team == 1:
                    current_team2_hp -= damage
                    log_entry = f"Team 1's {second['name']} attacks for {damage} damage!"
                else:
                    current_team1_hp -= damage
                    log_entry = f"Team 2's {second['name']} attacks for {damage} damage!"

                battle_log.append(log_entry)
                print(log_entry)

                # Check if the first Pokemon fainted
                if (second_team == 1 and current_team2_hp <= 0) or (second_team == 2 and current_team1_hp <= 0):
                    if second_team == 1:
                        log_entry = f"Team 2's {first['name']} fainted!"
                        team2_remaining -= 1
                        team2_index = (team2_index + 1) % len(valid_team2)
                        team1_score += 1
                    else:
                        log_entry = f"Team 1's {first['name']} fainted!"
                        team1_remaining -= 1
                        team1_index = (team1_index + 1) % len(valid_team1)
                        team2_score += 1

                    battle_log.append(log_entry)
                    print(log_entry)

            print(f"Team 1: {team1_remaining} remaining, Team 2: {team2_remaining} remaining")
            print()

            turn += 1

        # Battle results
        print("\nBattle Results:")

        if team1_remaining <= 0 and team2_remaining <= 0:
            print("It's a tie! Both teams fainted simultaneously.")
        elif team1_remaining <= 0:
            print("Team 2 wins!")
        elif team2_remaining <= 0:
            print("Team 1 wins!")
        else:
            # Determine winner by score if turn limit reached
            if team1_score > team2_score:
                print("Team 1 wins on points!")
            elif team2_score > team1_score:
                print("Team 2 wins on points!")
            else:
                print("It's a tie based on points!")

        print(f"Final Score - Team 1: {team1_score}, Team 2: {team2_score}")

        # Return battle outcome for reference
        return {
            'winner': 'Team 1' if team1_remaining > 0 and team2_remaining <= 0 or team1_score > team2_score else
                     'Team 2' if team2_remaining > 0 and team1_remaining <= 0 or team2_score > team1_score else 'Tie',
            'team1_score': team1_score,
            'team2_score': team2_score,
            'team1_remaining': team1_remaining,
            'team2_remaining': team2_remaining,
            'turns': turn - 1,
            'battle_log': battle_log
        }

    def _calculate_damage(self, attacker, defender):
        """Calculate simplified damage for battle simulation."""
        # Determine if physical or special attack is higher
        physical = attacker['stats']['attack'] > attacker['stats']['sp_attack']

        if physical:
            attack_stat = attacker['stats']['attack']
            defense_stat = defender['stats']['defense']
        else:
            attack_stat = attacker['stats']['sp_attack']
            defense_stat = defender['stats']['sp_defense']

        # Base damage calculation (simplified)
        base_damage = (attack_stat * 2 / 5) * (attack_stat / defense_stat)

        # Type effectiveness (simplified)
        type_multiplier = 1.0
        for attack_type in attacker['types']:
            for defense_type in defender['types']:
                attack_type_lower = attack_type.lower()
                defense_type_lower = defense_type.lower()

                # Some types might need normalization
                if attack_type_lower == 'fighting':
                    attack_type_lower = 'fight'
                if defense_type_lower == 'fighting':
                    defense_type_lower = 'fight'

                # Get effectiveness from our lookup table
                if attack_type_lower in self.type_effectiveness and defense_type_lower in self.type_effectiveness[attack_type_lower]:
                    effectiveness = self.type_effectiveness[attack_type_lower][defense_type_lower]
                    type_multiplier *= effectiveness

        # STAB (Same Type Attack Bonus)
        stab = 1.5

        # Calculate final damage
        damage = int(base_damage * type_multiplier * stab)

        # Apply randomness (80-100% of calculated damage)
        damage = int(damage * (0.8 + random.random() * 0.2))

        # Ensure damage is at least 1
        return max(1, damage)

    def create_interactive_ui(self):
        """Create an interactive UI for the Advanced Pokemon Team Builder."""
        # Create widgets
        header = widgets.HTML("<h1>Advanced Pokémon Team Builder</h1>")
        display(header)

        # Create team size slider
        team_size_slider = widgets.IntSlider(
            value=6,
            min=1,
            max=6,
            step=1,
            description='Team Size:',
            style={'description_width': 'initial'},
            layout=widgets.Layout(width='300px')
        )

        # Create team input fields
        team_inputs = []
        for i in range(6):  # Maximum 6 Pokemon
            text_input = widgets.Text(
                value='',
                placeholder=f'Enter Pokémon {i+1} name',
                description=f'Pokémon {i+1}:',
                disabled=False,
                style={'description_width': 'initial'},
                layout=widgets.Layout(width='300px')
            )
            team_inputs.append(text_input)

        # Function to update which inputs are enabled based on team size
        def update_team_size(change):
            for i, inp in enumerate(team_inputs):
                inp.disabled = i >= change['new']

        team_size_slider.observe(update_team_size, names='value')

        # Initially disable inputs based on default value
        update_team_size({'new': team_size_slider.value})

        # Create buttons
        button_layout = widgets.Layout(width='auto', margin='10px')

        generate_button = widgets.Button(
            description='Generate Counter Team',
            button_style='primary',
            tooltip='Generate a counter team',
            layout=button_layout
        )

        random_button = widgets.Button(
            description='Random Team',
            button_style='info',
            tooltip='Generate a random team',
            layout=button_layout
        )

        analyze_button = widgets.Button(
            description='Analyze Team',
            button_style='success',
            tooltip='Analyze team strengths and weaknesses',
            layout=button_layout
        )

        rate_button = widgets.Button(
            description='Rate Team',
            button_style='warning',
            tooltip='Rate team on various metrics',
            layout=button_layout
        )

        moves_button = widgets.Button(
            description='Recommend Moves',
            button_style='danger',
            tooltip='Recommend moves for each Pokémon',
            layout=button_layout
        )

        evolution_button = widgets.Button(
            description='Evolution Planner',
            button_style='success',
            tooltip='Suggest evolution paths',
            layout=button_layout
        )

        competitive_button = widgets.Button(
            description='Competitive Analysis',
            button_style='warning',
            tooltip='Analyze competitive viability',
            layout=button_layout
        )

        battle_button = widgets.Button(
            description='Simulate Battle',
            button_style='danger',
            tooltip='Simulate battle between teams',
            layout=button_layout
        )

        # Output area
        output = widgets.Output()

        # Get team function
        def get_team_from_inputs():
            user_team = []
            for i in range(team_size_slider.value):
                pokemon_name = team_inputs[i].value
                if pokemon_name:
                    pokemon = self.get_pokemon_by_name(pokemon_name)
                    if pokemon:
                        user_team.append(self.pokemon_to_dict(pokemon))

            if not user_team:
                with output:
                    print("Please enter at least one valid Pokémon name.")
                return None

            return user_team

        # Button callbacks
        def on_generate_click(b):
            with output:
                clear_output()

                user_team = get_team_from_inputs()
                if user_team:
                    # Display user team
                    self.display_team(user_team, "Your Team")

                    # Generate and display counter team
                    counter_team = self.generate_counter_team(user_team, team_size_slider.value)
                    self.display_team(counter_team, "Counter Team")

                    # Analyze matchup
                    self._analyze_matchup(user_team, counter_team)

        def on_random_click(b):
            # Generate random Pokemon team
            random_pokemon = random.sample(self.all_pokemon_names, team_size_slider.value)

            # Fill in the text inputs
            for i in range(team_size_slider.value):
                team_inputs[i].value = random_pokemon[i]

        def on_analyze_click(b):
            with output:
                clear_output()

                user_team = get_team_from_inputs()
                if user_team:
                    self.display_team(user_team, "Your Team")
                    self.analyze_team_weaknesses(user_team)

        def on_rate_click(b):
            with output:
                clear_output()

                user_team = get_team_from_inputs()
                if user_team:
                    self.display_team(user_team, "Your Team")
                    self.rate_team(user_team)

        def on_moves_click(b):
            with output:
                clear_output()

                user_team = get_team_from_inputs()
                if user_team:
                    self.display_team(user_team, "Your Team")
                    self.recommend_moves(user_team)

        def on_evolution_click(b):
            with output:
                clear_output()

                user_team = get_team_from_inputs()
                if user_team:
                    self.display_team(user_team, "Your Team")
                    self.suggest_evolutions(user_team)

        def on_competitive_click(b):
            with output:
                clear_output()

                user_team = get_team_from_inputs()
                if user_team:
                    self.display_team(user_team, "Your Team")
                    self.analyze_competitive_tiers(user_team)

        def on_battle_click(b):
            with output:
                clear_output()

                user_team = get_team_from_inputs()
                if user_team:
                    # Generate a counter team
                    counter_team = self.generate_counter_team(user_team, team_size_slider.value)

                    # Display both teams
                    self.display_team(user_team, "Your Team")
                    self.display_team(counter_team, "Counter Team")

                    # Simulate battle
                    self.simulate_battle(user_team, counter_team)

        # Connect button callbacks
        generate_button.on_click(on_generate_click)
        random_button.on_click(on_random_click)
        analyze_button.on_click(on_analyze_click)
        rate_button.on_click(on_rate_click)
        moves_button.on_click(on_moves_click)
        evolution_button.on_click(on_evolution_click)
        competitive_button.on_click(on_competitive_click)
        battle_button.on_click(on_battle_click)

        # Help text
        help_text = widgets.HTML("""
        <div style="background-color: #f8f9fa; padding: 10px; border-radius: 5px; margin-top: 20px; margin-bottom: 20px;">
        <h3>How to use the Advanced Pokémon Team Builder:</h3>
        <ol>
            <li>Set your desired team size using the slider</li>
            <li>Enter Pokémon names in the text fields (or click "Random Team")</li>
            <li>Use the buttons below to access different features:</li>
        </ol>
        <ul>
            <li><strong>Generate Counter Team</strong>: Create a team that counters your current team</li>
            <li><strong>Random Team</strong>: Fill the inputs with random Pokémon</li>
            <li><strong>Analyze Team</strong>: Identify team weaknesses and vulnerabilities</li>
            <li><strong>Rate Team</strong>: Get ratings for your team across various metrics</li>
            <li><strong>Recommend Moves</strong>: Get move suggestions for each Pokémon</li>
            <li><strong>Evolution Planner</strong>: See how your team could evolve</li>
            <li><strong>Competitive Analysis</strong>: Analyze your team's competitive viability</li>
            <li><strong>Simulate Battle</strong>: Run a battle simulation against a counter team</li>
        </ul>
        <p><em>Tip: The system will try to match partial or misspelled names.</em></p>
        </div>
        """)

        # Arrange buttons in rows
        button_row1 = widgets.HBox([random_button, generate_button, analyze_button, rate_button])
        button_row2 = widgets.HBox([moves_button, evolution_button, competitive_button, battle_button])

        # Arrange widgets
        input_box = widgets.VBox([team_size_slider] + team_inputs)

        display(widgets.VBox([input_box, help_text, button_row1, button_row2, output]))


# Run the Advanced Pokemon Team Builder
def run_advanced_pokemon_team_builder(csv_path):
    builder = AdvancedPokemonTeamBuilder(csv_path)
    builder.create_interactive_ui()

# Example usage
path = "/content/drive/MyDrive/Colab Notebooks/All Dataset for Python/pokemon.csv"
run_advanced_pokemon_team_builder(path)

Loading data from /content/drive/MyDrive/Colab Notebooks/All Dataset for Python/pokemon.csv...
Loaded 801 Pokémon successfully!
Advanced Pokémon Team Builder initialized successfully!


HTML(value='<h1>Advanced Pokémon Team Builder</h1>')